### Intro to Dataframes

In [ ]:
import pandas as pd

df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=['A','B','C'], index=['x','y','z'])

In [ ]:
df.describe()

### Loading in Dataframes from Files


In [ ]:
coffee = pd.read_csv("./warmup-data/coffee.csv")

results = pd.read_parquet("./data/results.parquet")

olympics = pd.read_excel("./data/olympics-data.xlsx")

bios = pd.read_csv("./data/bios.csv")


### Accessing data

In [ ]:
coffee.sample(10)

In [ ]:
coffee.loc[0:6,["Day","Units Sold"]]

In [ ]:
coffee.iloc[0:6,[0,2]] # Only using index

In [ ]:
coffee.loc[1:4,"Unit Solds"] = 10

coffee.head()

In [ ]:
coffee.at[0,"Unit Solds"]=100

In [ ]:
coffee.sort_values("Unit Solds",ascending=True)

coffee.head()

In [ ]:
for idx , row in coffee.iterrows():
    print(idx)
    print(row)
    print("\n")

### Filtering data

In [ ]:
bios.loc[bios['height_cm']>215,["name","height_cm"]]

In [ ]:
bios[(bios["height_cm"]>215)][["name","height_cm"]]

In [ ]:
bios[(bios["height_cm"]>215) & (bios["born_country"]=="USA")]

In [ ]:
bios[bios["name"].str.contains("Hristo",case=False)]

In [ ]:
# Top 5 tallest overall
bios.nlargest(5, 'height_cm')[['name', 'height_cm', 'born_country']]  # top 5 tallest rows


In [ ]:
# Tallest person in each country (per-group max)
bios[bios['height_cm'].eq(bios.groupby('born_country')['height_cm'].transform('max'))][['name', 'born_country', 'height_cm']]  # tallest per country

In [ ]:
# Filter by a set of countries
bios[bios['born_country'].isin(['USA', 'Germany', 'Bulgaria'])][['name', 'born_country', 'height_cm']]  # only selected countries


In [ ]:
# Exclude certain countries
bios[~bios['born_country'].isin({'USA', 'Canada'})][['name', 'born_country', 'height_cm']].head(10)  # not USA/Canada


In [ ]:
# Names that start with A or H (case-insensitive)
bios[bios['name'].str.contains(r'^(A|H)', case=False, na=False)][['name']].head(10)  # anchors at start


In [ ]:
# Duplicate names (possible homonyms)
bios[bios['name'].duplicated(keep=False)].sort_values('name')[['name', 'born_country', 'height_cm']]  # all duplicates


In [ ]:
# Data-quality check: names containing digits
bios[bios['name'].str.contains(r'\d', na=False)][['name']]

In [ ]:
# Shortest person in each country (per-group min via sort+drop_duplicates)
(bios.sort_values(['born_country', 'height_cm'])
     .drop_duplicates('born_country'))[['name', 'born_country', 'height_cm']]  # shortest per country
